In [1]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
import json
import numpy as np

import sys
sys.path.insert(0, "..")

from utils import read_pybullet_log

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
# load experimental parameters
with open('trial_params_dynamics.json', 'r') as f:
    trial_params = json.load(f)
trial_params = {int(k): v for k, v in trial_params.items()}

# load displacement log
displacement_data = np.load('dynamics_data.npy')

In [4]:
data_mu = {'mu': []}
mus = sorted(list(set(np.array(trial_params.values()[:-1])[:, 1])))
for mu in mus:
    data_mu['mu'].append('mu=%.2f' % mu)

forces = sorted(list(set(np.array(trial_params.values()[:-1])[:, 2])))
for force in forces:
    data_mu['F=%dN' % force] = []

for k, v in trial_params.items():
    if v[0] != 0.4:  # only plot for m=0.4kg
        continue
    data_mu['F=%dN' % v[2]].append(displacement_data[k])

data_mu

{'F=100N': [0.5253264014311408, 0.13458845783824333, 0.07640734109597792],
 'F=50N': [0.13470143308978333, 0.03336572013899238, 0.018649139170956303],
 'F=75N': [0.29980820391439367, 0.07561822400499, 0.042679278270234225],
 'mu': ['mu=0.10', 'mu=0.40', 'mu=0.70']}

In [5]:
source = ColumnDataSource(data=data_mu)
p = figure(x_range=data_mu['mu'], y_range=(0, 0.55), plot_height=250, title="x-displacement, m=0.4kg", toolbar_location=None, tools="")
loc = -0.25
colors=["#c9d9d3", '#718dbf', "#e84d60"]
for i,force in enumerate(forces):
    p.vbar(x=dodge('mu', loc, range=p.x_range), top='F=%dN'%force, width=0.2, source=source,
       color=colors[i], legend_label='F=%dN'%force)
    loc+=0.25
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "horizontal"

show(p)

In [6]:
data_mass = {'mass': []}
masses = sorted(list(set(np.array(trial_params.values()[:-1])[:, 0])))
for mass in masses:
    data_mass['mass'].append('mass=%.2f' % mass)

forces = sorted(list(set(np.array(trial_params.values()[:-1])[:, 2])))
for force in forces:
    data_mass['F=%dN' % force] = []

for k, v in trial_params.items():
    if v[1] != 0.4:  # only plot for mu=0.4
        continue
    data_mass['F=%dN' % v[2]].append(displacement_data[k])

data_mass

{'F=100N': [1.982066623871087, 0.13458845783824333, 0.04371564719604784],
 'F=50N': [0.5304249781201003, 0.03336572013899238, 0.010659155363780082],
 'F=75N': [1.1588561148752845, 0.07561822400499, 0.024404410050846276],
 'mass': ['mass=0.10', 'mass=0.40', 'mass=0.70']}

In [7]:
source = ColumnDataSource(data=data_mass)
p = figure(x_range=data_mass['mass'], y_range=(0, 2.1), plot_height=250, title="x-displacement, mu=0.4", toolbar_location=None, tools="")
loc = -0.25
colors=["#c9d9d3", '#718dbf', "#e84d60"]
for i,force in enumerate(forces):
    p.vbar(x=dodge('mass', loc, range=p.x_range), top='F=%dN'%force, width=0.2, source=source,
       color=colors[i], legend_label='F=%dN'%force)
    loc+=0.25
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.legend.location = "top_right"
p.legend.orientation = "horizontal"

show(p)

In [8]:
for force in forces:
    for i in range(len(data_mass['F=%dN' % force])):
        print(data_mass['F=%dN' % force][i] / data_mu['F=%dN' % force][i])
    print('****')

3.937782738855889
1.0
0.5715628622891281
****
3.865324896867001
1.0
0.5718093426117429
****
3.7730192475979982
1.0
0.5721393595038871
****


# Scenario: Apply impulse force at block center

## General Observations:
* Behaves as expected -- greater force causes greater displacement and this displacement is affected by mass of object and mu.
* There is an exponential decrease in displacement with increasing force as mass and mu are increased independently. Change is more pronounced when mass is varied.
* The proportionality constants in the last cell indicate a relationship between mass and mu that is independent of the force applied. The nature (linear, exponential, etc.) of this relationship needs to explored with more experimental trials. Does this match Newtonian dynamics?

TODO: There is a way to add anisotropic friction in pybullet to more closely mimic real-world dynamics.

Note: GitHub strips away JavaScript plotting elements so view [this version of notebook](https://nbviewer.jupyter.org/github/HIRO-group/bullet_panda/blob/master/src/bullet_panda/data/data_dynamics.ipynb) for charts.